# **Import libraries**

In [1]:
!pip install ultralytics keras-facenet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.1/660.1 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.5 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=e7f32164186c6078a37e8ef68b0259b6e0c4cf4079ff261ffa2365e8d115272b
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [2]:
from ultralytics import YOLO
import os
import cv2
from PIL import Image
import numpy as np
from keras.models import load_model
import random
from keras_facenet import FaceNet
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle
import pandas as pd
from google.colab.patches import cv2_imshow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load extra data**

##├── extra_data
###│   ├── < name_1 >
####│   │   ├── images
###│   ├── < name_2 >
####│   │   ├── images
###│   ├── < name_.. >
####│   │   ├── images


In [4]:
#load all file_paths of person to corresponding label
def get_all_image_name_per_person(path):
  all_name_files = os.listdir(path)
  return all_name_files

file_path = '/content/drive/MyDrive/Biometrics/HW2/extra_data'
all_folders = os.listdir(file_path)
list_ = {}
for folder in all_folders:
  path_folder = os.path.join(file_path,folder)
  all_files_per_person = get_all_image_name_per_person(path_folder)
  list_[folder] = [os.path.join(path_folder, file) for file in all_files_per_person]

# **Detect faces and embed faces for extra data**

In [5]:
def crop_image(original_image, coordinate):
    cropped_image = original_image[int(coordinate[1]):int(coordinate[3]), int(coordinate[0]):int(coordinate[2])]
    cropped_image = cv2.resize(cropped_image, (160,160))
    return cropped_image

In [6]:
###Load weights
YOLO_model = YOLO('/content/drive/MyDrive/Biometrics/HW2/yolov8n-face.pt')

WARNING ⚠️ /content/drive/MyDrive/Biometrics/HW2/yolov8n-face.pt appears to require 'omegaconf', which is not in ultralytics requirements.
AutoInstall will run now for 'omegaconf' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['omegaconf'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=b62d12d8c4c1c40d9a1fdecd7106d4c0c8a92172e5e1fe37fffa8c53d040a85c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_3e7dkr8/wheels/12/

In [7]:
def detect_faces(data):
  list_faces = list()
  labels = list()
  cnt = 0
  for person in data:
    for file in data[person]:
      img = cv2.imread(file)
      if img is None:
        continue
      face = YOLO_model(file, conf = 0.4)
      coordinate_s = face[0].cpu().boxes.xyxy.tolist()
      if len(coordinate_s) == 1:
        [x1, y1, x2, y2] = coordinate_s[0]
        cropped_image = crop_image(img, [x1, y1, x2, y2])
        list_faces.append(cropped_image)
        labels.append(person)
  return np.asarray(list_faces), np.asarray(labels)

In [8]:
extra_images, extra_labels = detect_faces(list_)


image 1/1 /content/drive/MyDrive/Biometrics/HW2/extra_data/Quy Tung/17a42fbf2120887ed131.jpg: 640x448 1 face, 51.0ms
Speed: 12.7ms preprocess, 51.0ms inference, 48.2ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 /content/drive/MyDrive/Biometrics/HW2/extra_data/Quy Tung/791fc85bc7c46e9a37d5.jpg: 640x544 1 face, 51.0ms
Speed: 3.0ms preprocess, 51.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /content/drive/MyDrive/Biometrics/HW2/extra_data/Quy Tung/4180b0ceb851110f4840.jpg: 640x480 (no detections), 50.9ms
Speed: 2.6ms preprocess, 50.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/drive/MyDrive/Biometrics/HW2/extra_data/Quy Tung/c95ab6b0be2f17714e3e.jpg: 640x480 1 face, 6.7ms
Speed: 2.8ms preprocess, 6.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/drive/MyDrive/Biometrics/HW2/extra_data/Quy Tung/287fd85b94c13d9f64d01.jpg: 640x480 1 face, 6.6ms
Speed: 2.8ms 

In [9]:
embedder = FaceNet()
extra_embeddings = embedder.embeddings(extra_images)

1/1 [==============================] - 4s 4s/step


In [10]:
train_embedding = np.load('/content/drive/MyDrive/Biometrics/HW2/face_embedding/train_embeddings.npy')
train_labels = np.load('/content/drive/MyDrive/Biometrics/HW2/data/train_labels.npy')

In [11]:
train_embeddings = np.concatenate((train_embedding, extra_embeddings))
train_labels = np.concatenate((train_labels, extra_labels))

# **Train**

In [13]:
in_encoder = Normalizer()
emdTrainX_norm = in_encoder.transform(train_embedding)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_labels)
trainy_enc = out_encoder.transform(train_labels)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(emdTrainX_norm, trainy_enc)

SVC(kernel='linear', probability=True)

In [14]:
all_names =  out_encoder.inverse_transform([i for i in range(0, len(set(train_labels)))])

In [15]:
###save for further use
np.save('/content/drive/MyDrive/Biometrics/HW2/all_names.npy',all_names)
support_vectors_file = '/content/drive/MyDrive/Biometrics/HW2/extra_SVM.pickle'
pickle.dump(model, open(support_vectors_file, "wb"))

# **Webcam**
## *- Test on local machine*

In [ ]:
##load model
YOLO_model = YOLO('yolov8n-face.pt')
embedder = FaceNet()
SVM_model = pickle.load(open('extra_SVM.pickle', 'rb'))

In [ ]:
def predict(img, model):
  samples = np.expand_dims(img, axis=0)
  vector_embedding = embedder.embeddings(samples)
  yhat_prob = model.predict_proba(vector_embedding)
  return np.argmax(yhat_prob, axis=1)

In [ ]:
all_names = np.load('all_names.npy',allow_pickle=True)
all_names = list(all_names.tolist())

In [ ]:
# Khởi tạo webcam
cap = cv2.VideoCapture(0)

while True:
    # Đọc khung hình từ webcam
    ret, frame = cap.read()

    face = YOLO_model(frame, conf = 0.5)

    coordinate = face[0].cpu().boxes.xyxy.tolist()
    if len(coordinate) == 1:
        [x1, y1, x2, y2] = coordinate[0]
        cropped_image = crop_image(frame, [x1, y1, x2, y2])
        result = predict(cropped_image, SVM_model)
        name = all_names[result[0]]
    # Hiển thị tên
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, name, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Hiển thị video từ webcam
    cv2.imshow('Face Recognition', frame)

    # Thoát khỏi vòng lặp nếu người dùng nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng webcam và đóng cửa sổ hiển thị
cap.release()
cv2.destroyAllWindows()